In [ ]:
from google.colab import drive
import os
drive.mount('/content/drive')
DATA_DIR='/content/drive/MyDrive/Colab Notebooks/mesh2matrix/mesh'
#old DATA_DIR='/content/drive/MyDrive/mesh'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
os.chdir('/content/drive/MyDrive/Colab Notebooks/mesh2matrix')

In [ ]:
!pip install -r MeSH2Matrix/requirements.txt
!pip install livelossplot --quiet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
import os
import matplotlib.pyplot as plt
import pandas as pd
import torch
import numpy as np
from sklearn.model_selection import train_test_split
import torchtext
from torch.utils.data import Dataset,DataLoader
from torchtext.legacy.data import Field, TabularDataset, BucketIterator, Iterator
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import torch.optim as optim
import torch.nn.functional as F
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix,f1_score
from livelossplot import PlotLosses


device = torch.device('cuda' if True and torch.cuda.is_available() else 'cpu')

NUM_CLASSES=195
BATCH_SIZE=128

MeSH Dataset Class...

In [ ]:
class MESHDataset(Dataset):
    def __init__(self,numpy_file,label_file):
     
      try:
        self.data= np.load(numpy_file)
        self.labels = np.load(label_file)
      except Exception as err:
        raise Exception(f'ERROR OPENING FILES: {numpy_file} | {label_file}. See Error below. \n {err}')  
       
    def __len__(self):
        return self.data.shape[0]
    def __getitem__(self, idx):
        #Get the element with `idx`
        #Output an 89*89 matrix
        return np.expand_dims(self.data[idx], axis=0), self.labels[idx]
      

Baseline Model...

In [ ]:
class InceptionModel(nn.Module):

    def __init__(self, matrix_size=89):
      #Idea
      """
      ConvNet -> ConvNet -> Flatten ->  Dense => Class 
      """
      super(InceptionModel, self).__init__()
      self.layer1 = nn.Sequential(
          nn.Conv2d(1, 128, kernel_size=3, stride=1),
          nn.BatchNorm2d(128),
          nn.ReLU(),
          nn.MaxPool2d(kernel_size=2, stride=2),
          nn.Conv2d(128,64, kernel_size=3, stride=1),
          nn.BatchNorm2d(64),
          nn.ReLU(),
          nn.MaxPool2d(kernel_size=2, stride=2),
          nn.Conv2d(64,32, kernel_size=3, stride=1),
          nn.BatchNorm2d(32),
          nn.ReLU(),
          nn.MaxPool2d(kernel_size=2, stride=2)
          )
      self.layer2 = nn.Sequential(
          nn.Conv2d(32, 16, kernel_size=3, stride=1),
          nn.BatchNorm2d(16),
          nn.ReLU(),
          nn.MaxPool2d(kernel_size=2, stride=2)
        )
      
      self.drop_out = nn.Dropout()
      self.fc = nn.Sequential(
          #nn.Linear(12800, 5000),
          #nn.ReLU(),
          #nn.Linear(5000, 1000),
          #nn.ReLU(),
          nn.Linear(2592,1024),
          nn.ReLU(),
          nn.Linear(1024,500),
          nn.ReLU(),
          nn.Linear(500,NUM_CLASSES)
      )
    def forward(self, x):

        #Expecting `x` to be an 89*89 matrix. 
        #print(f'X original shape: {x.shape}')
        x=self.layer1(x)
        #x = self.layer2(x)
        x = x.reshape(x.size(0), -1)
        #print(f'Flattened X has shape: {x.shape}')
        x=self.drop_out(x)
        x=self.fc(x)
        #out = F.log_softmax(x) #Don't use softmax here since we use torch.nn.CrossEntropyLoss. 
        return x

Creating the dataloaders...

In [ ]:
train_dataset = MESHDataset(os.path.join(DATA_DIR,'output/train.npy'),os.path.join(DATA_DIR,'output/train_labels.npy'))
dev_dataset = MESHDataset(os.path.join(DATA_DIR,'output/dev.npy'),os.path.join(DATA_DIR,'output/dev_labels.npy'))
test_dataset = MESHDataset(os.path.join(DATA_DIR,'output/test.npy'),os.path.join(DATA_DIR,'output/test_labels.npy'))


train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
dev_dataloader = DataLoader(dev_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)


In [ ]:
print(f'train shape: {len(train_dataset)}')
print(f'dev shape: {len(dev_dataset)}')
print(f'test shape: {len(test_dataset)}')

print(f'train shape: {train_dataset.__len__()}')
print(f'dev shape: {dev_dataset.__len__()}')
print(f'test shape: {test_dataset.__len__()}')

train shape: 33457
dev shape: 13012
test shape: 9294
train shape: 33457
dev shape: 13012
test shape: 9294


In [ ]:
len(train_dataset)+len(dev_dataset)

46469

Creating the model...


In [ ]:
model = InceptionModel().to(device)
model

InceptionModel(
  (layer1): Sequential(
    (0): Conv2d(1, 128, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(128, 64, kernel_size=(3, 3), stride=(1, 1))
    (5): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(64, 32, kernel_size=(3, 3), stride=(1, 1))
    (9): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU()
    (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(32, 16, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(k

Training Configuration...

In [ ]:
import sys
def compute_accuracy(pred,target):  
  acc = (target == pred.argmax(dim=1)).float().detach().numpy()
  return float(100 * acc.sum() / len(acc))


In [ ]:
#Define criterion: Categorical cross entropy
criterion = nn.CrossEntropyLoss()
#Define optimizer. For now, Adam
optimizer = optim.Adam(model.parameters(), lr=1e-5)

PRINT_FREQ = 5
DO_VALIDATION_STEP=30
NUM_EPOCHS=1500
PATIENCE=50

PATH=os.path.join(DATA_DIR,'output/best_model_cnn_best_128')

if os.path.isfile(PATH):
  model.load_state_dict(torch.load(PATH))

best_acc = 0

liveloss = PlotLosses()
patience=0

'''
for epoch in range(NUM_EPOCHS):
  for i,(features, label) in enumerate(train_dataloader):
    features, label = features.to(device), label.to(device)
    optimizer.zero_grad()
    pred = model(features.float())
    loss = criterion(pred,label)

   
    
    loss.backward()
    optimizer.step()

    if i%DO_VALIDATION_STEP==0:
      #Do validation
      model.eval()
      val_losses=[]
      val_acc_list=[]
      for val_features, val_label in dev_dataloader:
         val_features, val_label = val_features.to(device), val_label.to(device)
         val_pred = model(val_features.float())
         val_loss = criterion(val_pred,val_label)
         val_losses.append(val_loss.item())
         val_acc_list.append(compute_accuracy(val_pred.cpu(),val_label.cpu()))

      val_loss_ = np.mean(val_losses) 
      val_acc = np.mean(val_acc_list)
      if val_acc> best_acc:
        best_acc=val_acc
        patience=0
        #Save model weights
        torch.save(model.state_dict(), PATH)
      else:
        patience+=1
      if patience >= PATIENCE:
        break    
      
      logs={'loss':loss.item(),'val_loss':val_loss_,'val_accuracy':val_acc}
      liveloss.update(logs)
      liveloss.send()  
      model.train()  
'''

"\nfor epoch in range(NUM_EPOCHS):\n  for i,(features, label) in enumerate(train_dataloader):\n    features, label = features.to(device), label.to(device)\n    optimizer.zero_grad()\n    pred = model(features.float())\n    loss = criterion(pred,label)\n\n   \n    \n    loss.backward()\n    optimizer.step()\n\n    if i%DO_VALIDATION_STEP==0:\n      #Do validation\n      model.eval()\n      val_losses=[]\n      val_acc_list=[]\n      for val_features, val_label in dev_dataloader:\n         val_features, val_label = val_features.to(device), val_label.to(device)\n         val_pred = model(val_features.float())\n         val_loss = criterion(val_pred,val_label)\n         val_losses.append(val_loss.item())\n         val_acc_list.append(compute_accuracy(val_pred.cpu(),val_label.cpu()))\n\n      val_loss_ = np.mean(val_losses) \n      val_acc = np.mean(val_acc_list)\n      if val_acc> best_acc:\n        best_acc=val_acc\n        patience=0\n        #Save model weights\n        torch.save(model

Compute test metrics... 

In [ ]:
from sklearn.metrics import accuracy_score
def compute_accuracy(pred, target):
  return target.detach().numpy(), pred.argmax(-1).detach().numpy()

In [ ]:
model.eval()

labels, preds = [], []
for test_features, test_label in test_dataloader:
  # print(test_features.size())
  test_features, test_label = test_features.to(device), test_label.to(device)
  test_pred = model(test_features.float())
  true, predictions = compute_accuracy(test_pred.cpu(),test_label.cpu())
  labels += true.tolist()
  preds += predictions.tolist()
print(f'Best CNN Model Accuracy with 195 classes:{accuracy_score(labels, preds)}')
print("Best CNN Model Weighted F1-Score with 195 classes: {}".format(f1_score(labels, preds, average='weighted')))

Best CNN Model Accuracy with 195 classes:0.7048633527006671
Best CNN Model Weighted F1-Score with 195 classes: 0.6617880913997406


# Confusion Matrix

In [ ]:
confusion= confusion_matrix(labels, preds)
print(confusion)

[[ 0  0  0 ...  0  0  0]
 [ 0 52  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 ...
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]]


In [ ]:
len(np.unique(labels))

125

In [ ]:
confusion.shape

(128, 128)